In [1]:
import os
import re
import tabula
import pandas as pd
import numpy as np

In [2]:
# Set the JAVA_HOME environment variable to the Java installation directory
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk/libexec/openjdk.jdk"

In [3]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
%pip install -q tabula-py
%pip install requests beautifulsoup4
%pip install openpyxl
%pip install selenium
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### **Importing**

Download the source code for CASURECO II's webpage.

In [5]:
import requests

url = 'https://www.casureco2.com.ph/support/rates?page=1'
response = requests.get(url)

if response.status_code == 200:
    with open('generation_charge.html', 'w', encoding='utf-8') as file:
        file.write(response.text)
    print("Webpage downloaded and saved as 'generation_charge.html'.")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

Webpage downloaded and saved as 'generation_charge.html'.


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import os
import time

# Set up Selenium WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get('https://www.casureco2.com.ph/support/rates?page=1')

# Create a directory for storing PDFs if it doesn't exist
download_dir = 'pdf downloads'
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

# Function to download PDF
def download_pdf(pdf_url, save_path):
    response = requests.get(pdf_url)
    with open(save_path, 'wb') as file:
        file.write(response.content)
    print(f"Downloaded: {save_path}")

# Function to process the current page
def process_current_page():
    # Wait for the table to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'biddings_table')))
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table', {'id': 'biddings_table'})
    rows = table.find('tbody').find_all('tr')

    for row in rows:
        # Find the "View PDF" button and click it
        view_pdf_button = row.find('button', {'data-dropdown-toggle': True})
        if view_pdf_button:
            view_pdf_button_id = view_pdf_button['id']
            driver.find_element(By.ID, view_pdf_button_id).click()

            # Wait for the dropdown to be visible
            dropdown_id = f'dropdown{view_pdf_button_id.split("dropdownDefaultButton")[-1]}'
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, dropdown_id)))

            # Get the "Generation Charge" link and download the PDF if not disabled
            dropdown = driver.find_element(By.ID, dropdown_id)
            gen_charge_link_element = dropdown.find_element(By.LINK_TEXT, 'Generation Charge')
            gen_charge_link = gen_charge_link_element.get_attribute('href')
            
            # Extract month and year
            month = row.find_all('th')[0].text.strip()
            year = row.find_all('th')[1].text.strip()

            # Print the contents of gen_charge_link and its attributes
            print(f"Link href: {gen_charge_link}")
            print(f"Link attributes: {gen_charge_link_element.get_attribute('outerHTML')}")
            
            if 'disabled' not in gen_charge_link_element.get_attribute('outerHTML'):
                save_path = os.path.join(download_dir, f'{month}_{year}_Generation_Charge.pdf')
                download_pdf(gen_charge_link, save_path)
            else:
                print(f"Link for {month} {year} is disabled, skipping download.")

            # Close the dropdown by clicking again on "View PDF"
            driver.find_element(By.ID, view_pdf_button_id).click()

# Function to check if we are on the last page
def is_last_page():
    try:
        results_text = driver.find_element(By.XPATH, '//p[@class="text-sm text-gray-700 dark:text-white leading-5"]').text
        parts = results_text.split()
        showing_to = int(parts[3])  # This should be the second number in the text "Showing X to Y of Z results"
        total = int(parts[5])       # This should be the total number of results
        return showing_to == total
    except Exception as e:
        print(f"Error checking last page: {e}")
        return False

# Loop through pages
while True:
    process_current_page()
    
    # Check if there is a next page
    if is_last_page():
        break
    
    try:
        next_button = driver.find_element(By.XPATH, '/html/body/div/div[1]/main/section/div[2]/div/div[2]/nav/div[2]/div[2]/span/a[2]')
        next_button.click()
        time.sleep(2)  # Wait for the next page to load
    except Exception as e:
        print(f"An error occurred: {e}")
        break

# Close the driver
driver.quit()

Link href: https://www.casureco2.com.ph/public/public/rates_images/JULY%20GEN-MIX%20-%20Copy_1721352610.pdf
Link attributes: <a href="https://www.casureco2.com.ph/public/public/rates_images/JULY GEN-MIX - Copy_1721352610.pdf" class="block px-4 py-2 hover:bg-gray-100 dark:hover:bg-gray-600 dark:hover:text-white">
                                                                Generation Charge
                                                            </a>
Downloaded: pdf downloads/July_2024_Generation_Charge.pdf
Link href: https://www.casureco2.com.ph/public/public/rates_images/JUNE%20GENERATION%20CHARGE_1718848441.pdf
Link attributes: <a href="https://www.casureco2.com.ph/public/public/rates_images/JUNE GENERATION CHARGE_1718848441.pdf" class="block px-4 py-2 hover:bg-gray-100 dark:hover:bg-gray-600 dark:hover:text-white">
                                                                Generation Charge
                                                            </a>
Downloaded: pdf 

### **Processing**

In [49]:
df = tabula.read_pdf("pdf downloads/June_2024_Generation_Charge.pdf", stream=True, pages='all')[0]

In [50]:
df

,Unnamed: 0,Unnamed: 1,(A),(B),(C ),(D),E (E= C + D),F (F=E/A)
0,NaN,NaN,NaN,NaN,NaN,Other,Total Generation Cost,NaN
1,NaN,NaN,NaN,Energy Share,Basic Generation Cost,NaN,NaN,Generation Rate
2,NaN,Source,kWh Purchased,NaN,NaN,Charges/Adjustments,for the month,NaN
3,NaN,NaN,NaN,(%),(Php),NaN,NaN,(Php/kWh)
4,NaN,NaN,NaN,NaN,NaN,(Php),(Php),NaN
5,Power Suppliers /IPPs,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,PSALM,NaN,"4,052,294",9%,"25,336,902.74","99,281.21","25,436,183.95",6.2770
7,Net Metering,NaN,"8,861",0.02%,"59,786.94",NaN,"59,786.94",6.7472
8,Wholesale Electricity Spot Market (WESM),NaN,"43,554,260",91%,"88,610,362","74,068","88,684,429.46",2.0362
9,Sub-Total,NaN,"47,615,415",100%,"114,007,051.59","173,348.76","114,180,400.35",NaN


In [51]:
# Define the keywords to search for in row values
keywords = ['Suppliers', ',', '.', '%']

# Initialize a list to store the columns to keep
columns_to_keep = []

# First column to keep: Find the column containing the word 'Suppliers'
suppliers_col = df.columns[df.apply(lambda col: col.astype(str).str.contains('Suppliers', case=False, na=False)).any()]
if not suppliers_col.empty:
    suppliers_col_name = suppliers_col[0]
    columns_to_keep.append(suppliers_col_name)

    # Second column to keep: Find the first column with a comma, excluding the 'Suppliers' column
    comma_col = df.columns[
        df.apply(lambda col: col.astype(str).str.contains(',', case=False, na=False)).any()
    ]
    if not comma_col.empty:
        comma_col_name = comma_col[0]
        
        if comma_col_name != suppliers_col_name:
            columns_to_keep.append(comma_col_name)
        else:
            comma_col_name = comma_col[1]
            columns_to_keep.append(comma_col_name)

        # Third column to keep: Find the column with a percentage symbol, excluding the previous columns
        percent_cols = df.columns[
            df.apply(lambda col: col.astype(str).str.contains('%', case=False, na=False)).any()
        ]
        
        if not percent_cols.empty:
            percent_col_name = percent_cols[0]
            if percent_col_name not in columns_to_keep:
                columns_to_keep.append(percent_col_name)

        # Fourth column to keep: Find the last column with a period, excluding the previous columns
        period_cols = df.columns[
            df.apply(lambda col: col.astype(str).str.contains('.', case=False, na=False)).any()
        ]
        
        if not period_cols.empty:
            period_cols = [col for col in period_cols if col not in columns_to_keep]
            if period_cols:
                last_period_col = period_cols[-1]
                if last_period_col not in columns_to_keep:
                    columns_to_keep.append(last_period_col)

# Drop duplicates to ensure unique columns
columns_to_keep = list(dict.fromkeys(columns_to_keep))

# Slice the DataFrame to keep only the relevant columns
df_sliced_columns = df[columns_to_keep]

# Reorder columns to ensure 'Energy Share' is the third column
if 'Energy Share' in df_sliced_columns.columns:
    columns_order = [col for col in columns_to_keep if col != 'Energy Share'] + ['Energy Share']
    df_sliced_columns = df_sliced_columns[columns_order]

# Rename columns for consistency if the length matches
if len(df_sliced_columns.columns) == 4:
    df_sliced_columns.columns = ["Power Supplier", "kWh", "Energy Share", "Average Generation Cost"]

df_sliced_columns

,Power Supplier,kWh,Energy Share,Average Generation Cost
0,NaN,NaN,NaN,NaN
1,NaN,NaN,Energy Share,Generation Rate
2,NaN,kWh Purchased,NaN,NaN
3,NaN,NaN,(%),(Php/kWh)
4,NaN,NaN,NaN,NaN
5,Power Suppliers /IPPs,NaN,NaN,NaN
6,PSALM,"4,052,294",9%,6.2770
7,Net Metering,"8,861",0.02%,6.7472
8,Wholesale Electricity Spot Market (WESM),"43,554,260",91%,2.0362
9,Sub-Total,"47,615,415",100%,NaN


In [52]:
# Sample DataFrame for demonstration (replace this with your actual DataFrame)
# df_sliced_columns = pd.read_csv('your_data.csv')  # Load your actual DataFrame

# Find the index of rows containing specific keywords in the first column
suppliers_index = df_sliced_columns.index[df_sliced_columns.iloc[:, 0].str.contains('Suppliers', case=False, na=False)]
sub_total_index = df_sliced_columns.index[df_sliced_columns.iloc[:, 0].str.contains('Sub-Total', case=False, na=False)]
#total_generation_cost_index = df_sliced_columns.index[df_sliced_columns.iloc[:, 0].str.contains('TOTAL GENERATION COST', case=False, na=False)]

# Ensure we have found these rows
if not suppliers_index.empty and not sub_total_index.empty:
    suppliers_index = suppliers_index[0]
    sub_total_index = sub_total_index[0]

    # Slice the DataFrame from after "Suppliers" row to "Sub-Total" row (inclusive)
    df_filtered = df_sliced_columns.loc[suppliers_index + 1: sub_total_index]

df_filtered

,Power Supplier,kWh,Energy Share,Average Generation Cost
6,PSALM,"4,052,294",9%,6.2770
7,Net Metering,"8,861",0.02%,6.7472
8,Wholesale Electricity Spot Market (WESM),"43,554,260",91%,2.0362
9,Sub-Total,"47,615,415",100%,NaN


In [56]:
# Ensure that df_filtered is a copy to avoid SettingWithCopyWarning
df_filtered = df_filtered.copy()

# Find the row with 'TOTAL' and extract the value from 'Column Average'
total_row = df_sliced_columns[df_sliced_columns['Power Supplier'].str.contains('TOTAL GENERATION COST', case=False, na=False)]
generation_charge = total_row['Average Generation Cost'].values[0] if not total_row.empty else None

# Add the new column "Generation Charge" with the extracted value
df_filtered.loc[:, 'Generation Charge'] = generation_charge

# Reset the index of the final dataframe
df_filtered.reset_index(drop=True, inplace=True)

df_filtered

,Power Supplier,kWh,Energy Share,Average Generation Cost,Generation Charge
0,PSALM,"4,052,294",9%,6.2770,2 .7423
1,Net Metering,"8,861",0.02%,6.7472,2 .7423
2,Wholesale Electricity Spot Market (WESM),"43,554,260",91%,2.0362,2 .7423
3,Sub-Total,"47,615,415",100%,NaN,2 .7423
